# Kaggle-Freesound-audio-tagging

https://www.kaggle.com/c/freesound-audio-tagging

Решение с небольшим изменением параметров дает 0.8 очков на Public Leaderboard, Ваша задача преодолеть 0.9.


Проверьте, что папка с данными лежит по пути `DATADIR`, архивы распакованы.
Потестировано на версиях python 2.7/3.5/3.6, tf>=1.4. 

Код под одну карточку, так что на машинах с несколькими запускайте с ограничениями `CUDA_VISIBLE_DEVICES=x jupyter notebook ...` или модифицируйте код.

Более гибкое решение можно описать в файлах (см. README.md)


In [ ]:
from __future__ import print_function
import os
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # закомментируйте на время отладки

import pandas as pd
import numpy as np
import json

import tensorflow as tf
from tensorflow.contrib import layers
from sklearn.model_selection import train_test_split
from tensorflow.contrib.learn.python.learn.learn_io.generator_io import generator_input_fn

import utils


# сначала проверьте пути
DATADIR='/data/kaggle-freesound/'
OUTDIR = './runs/'

try:
    os.makedirs(OUTDIR)
except OSError:
    pass


# tf.logging.set_verbosity(tf.logging.ERROR)  # закомментируйте на время отладки


Проведем сначала небольшой EDA касательно данных

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm_notebook
from scipy.io import wavfile
from glob import glob

In [ ]:
%%time
train_files = glob(os.path.join(DATADIR, 'audio_train/*wav'))
test_files = glob(os.path.join(DATADIR, 'audio_test/*wav'))
train_lens = [len(wavfile.read(_)[1]) for _ in tqdm_notebook(train_files)]
test_lens = [len(wavfile.read(_)[1]) for _ in tqdm_notebook(test_files)]
sns.distplot(train_lens, kde=False, label='train')
sns.distplot(test_lens, kde=False, label='test')

min(train_lens), min(test_lens)

**Обратите внимание, что у файлов разная длина. А в тесте есть пустые записи.**

Это необходимо будет учесть при написании загрузчика

В качестве простого решения на сетках предлагается нарезать записи на фрагменты одинакового размера, преобразовать их в спектрограммы.
Из одномерного сигнала (timesteps, ) получается спектрограмма (time_bins, freq_bins) -- комплекснозначная матрица.
Обычный подход -- это выделить модуль и угол комплексных чисел, немного преобразовать и склеить обратно в массив. 

Аудиозаписи в задаче лежат в формате wav -- это просто одномерные сигналы в формате int16.
Для дальнейшей работы их надо будет перевести в float32:

In [ ]:
def _read(fname):
    _, wav = wavfile.read(fname)
    wav = wav.astype(np.float32) / np.iinfo(np.int16).max
    return wav

In [ ]:
sample = _read(train_files[0])

In [ ]:
import tensorflow as tf
from tensorflow.contrib import signal

tf.reset_default_graph()
x = tf.placeholder(np.float32, [None])

# обратите внимание на параметры -- их можно варьировать, получая больше частот.
# обратите внимание, что исходные сигналы имеют sampling rate 44100
specgram = signal.stft(x, 800, 400)  # [time_bins, freq_bins]

phase = tf.angle(specgram) / np.pi  # приводим угол к [-1, 1]
amp = tf.log1p(tf.abs(specgram))  # одно из обычных преобразований для амплитуды
with tf.Session() as sess:
    v, w = sess.run([amp, phase], feed_dict={x: _read(train_files[2])})


In [ ]:
# здесь среднее вычитается для наглядности картинок
plt.figure(figsize=(8, 12))
plt.title('amp')
sns.heatmap(v - v.mean(axis=0), robust=True, center=0)
plt.show()

plt.figure(figsize=(8, 12))
plt.title('phase')
sns.heatmap(w - w.mean(axis=0), robust=True, center=0)
plt.show()

In [ ]:
Угол не выглядит слишком полезным, а на аплитуде мало высоких частот.

In [ ]:
V, W = [], []
with tf.Session() as sess:
    for fname in tqdm_notebook(train_files):
        v = sess.run(amp, feed_dict={x: _read(fname)})
        V.append(np.mean(v, axis=0))
        W.append(np.std(v, axis=0))
        
        
V = np.array(V)
W = np.array(W)

In [ ]:
plt.figure()
plt.plot(np.mean(V, axis=0), label='mean amp')
plt.plot(np.mean(W, axis=0), label='std amp')

plt.xlabel('freq bin')
plt.legend()
plt.show()

Вы можете перевзвесить спектрограмму так чтобы высокие частоты имели больший вес или просто сместить среднее на посчитанные кривые.

Придумывайте гипотезы и экспериментируйте.

In [ ]:
# прочитаем данные и разобьем выделим трейн и валидацию
df = pd.read_csv(os.path.join(DATADIR, 'train.csv'))
labels = sorted(set(df.label.values))

label2id = {label: i for i, label in enumerate(labels)}
id2label = {i: label for label, i in label2id.items()}

df['label'] = [label2id[_] for _ in df.label.values]
df['fname'] = [
    os.path.join(DATADIR, 'audio_train', _) for _ in df.fname.values]

# todo: разберитесь с форматом входных данных, потюньте процедуру разбиения
# можно добавить фолды, балансировать классы или разбивать по флагу ручной разметки
idx = np.arange(len(df))
idx_train, idx_val = train_test_split(
    idx, test_size=0.33, random_state=2018, shuffle=True)
df_train, df_val = df.iloc[idx_train], df.iloc[idx_val]

In [ ]:
# далее займемся сеткой
tf.reset_default_graph()

# Опишем сетку (тушку, body, feature extractor).
# На вход приходит _картинка_ размером [?, ?, ?, 2] 
#  количество частот мы можем менять параметрами stft
#  количество шагов по времени зависит от stft и длины самой записи
#  канал с углом можно выбросить
def baseline(x, params, is_training):
    # это общие параметры для сверточных слоев, мы будем передавать их явно:
    afn = dict(
        normalizer_fn=layers.batch_norm,
        normalizer_params=dict(is_training=is_training),
    )

    # baseline всего на три слоя сверток и пулингов. Поменяйте на свою
    for i in range(3):
        if is_training:
            x = tf.nn.dropout(x, 0.9)
        x = layers.conv2d(x, 16 * (2 ** i), (3, 11), **afn)
        x = layers.max_pool2d(x, 2, 2)
        
    # GAP (Global Average Polling) уберет пространственные размерности и оставит только каналы
    gap = tf.reduce_mean(x, axis=[1, 2], keep_dims=True)
    gap = tf.nn.dropout(gap, params.keep_prob if is_training else 1.0)

    # вместо полносвязного слоя удобно взять свертку 1х1 на нужное количество классов
    x = tf.layers.conv2d(gap, params.num_classes, 1, activation=None)
    # тушка возвращает логиты
    return tf.squeeze(x, [1, 2])

In [ ]:
# Теперь опишем модельку:
#   features -- словарь с входными тензорами, 
#   labels -- тензор с метками, 
#   mode -- один из трех вариантов tf.estimator.ModeKeys.TRAIN/EVAL/PREDICT
#   params -- набор параметров, который мы сформируем позже
#
#   функция должна вернуть правильно заполненный tf.estimator.EstimatorSpec(**specs), 
#     см документацию и комменты в исходном коде, если хотите разобраться глубже

def model_handler(features, labels, mode, params, config):
    # эта функция делает три разных версии вычислительного графа в зависимости от параметра mode
    # общим остается преобразование сигнала в спектрограмму и проход сетки
    # в тренировочном варианте к графу добавляются тренировочные вершины
    # в валидационном подсчет разных метрик
    extractor = tf.make_template(
        'extractor', baseline,
        create_scope_now_=True,
    )

    wav = features['wav']  # здесь будет тензор [bs, timesteps]
    specgram = signal.stft(wav, 400, 160)  # здесь комплекснозначный тензор [bs, time_bins, freq_bins]

    phase = tf.angle(specgram) / np.pi
    amp = tf.log1p(tf.abs(specgram))

    x = tf.stack([amp, phase], axis=3)  # здесь почти обычная картинка  [bs, time_bins, freq_bins, 2]
    x = tf.to_float(x)

    logits = extractor(x, params, mode == tf.estimator.ModeKeys.TRAIN)
    predictions = tf.nn.softmax(logits)

    if mode == tf.estimator.ModeKeys.TRAIN:
        loss = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                labels=labels, logits=logits)
        )

        # todo: обязательно попробуйте другие варианты изменния lr
        def _learning_rate_decay_fn(learning_rate, global_step):
            return tf.train.exponential_decay(
                learning_rate, global_step, decay_steps=10000, decay_rate=0.99)

        train_op = tf.contrib.layers.optimize_loss(
            loss=loss,
            global_step=tf.contrib.framework.get_global_step(),
            learning_rate=params.learning_rate,
            optimizer=lambda lr: tf.train.AdamOptimizer(lr),  # оптимизатор точно стоит потюнить
            learning_rate_decay_fn=_learning_rate_decay_fn,
            clip_gradients=params.clip_gradients,
            variables=tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES))

        specs = dict(
            mode=mode,
            loss=loss,
            train_op=train_op,
        )

    if mode == tf.estimator.ModeKeys.EVAL:
        acc, acc_op = tf.metrics.accuracy(labels, tf.argmax(logits, axis=-1))
        # см https://www.kaggle.com/c/freesound-audio-tagging#evaluation
        # метрика оценки mean average precision at 3 (MAP@3)
        map3, map3_op = tf.metrics.sparse_average_precision_at_k(
            tf.cast(labels, tf.int64), predictions, 3)
        loss = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                labels=labels, logits=logits)
        )
        specs = dict(
            mode=mode,
            loss=loss,
            eval_metric_ops={
                "MAP@1": (acc, acc_op),
                "MAP@3": (map3, map3_op),
            }
        )

    if mode == tf.estimator.ModeKeys.PREDICT:
        predictions = {
            # здесь можно пробрасывать что угодно
            'predictions': predictions,  # весь вектор предсказаний
            'top3': tf.nn.top_k(predictions, 3)[1],  # топ-3 метки
            'prediction': tf.argmax(predictions, 1),  # топовая метка
            'fname': features['fname'],  # имя файла, будет удобно работать во время предсказаний на тесте
        }
        specs = dict(
            mode=mode,
            predictions=predictions,
        )
    return tf.estimator.EstimatorSpec(**specs)

In [ ]:
# создадим очередную папку для модельки
outdir = utils.get_new_model_path(OUTDIR)

# собираем вместе параметры
params = dict(
    outdir=outdir,
    seed=2018,
    datadir=DATADIR,
    batch_size=32,
    keep_prob=0.8,
    learning_rate=3e-4,
    clip_gradients=15.0,
    num_classes=len(label2id),
    train_steps=10000, # количество шагов в тренировке
    signal_len=8000,  # размер фрагмента для классификации
)


hparams = tf.contrib.training.HParams(**params)
# на всякий случай сохраним конфиг и словарь меток в папку с запуском
with open(os.path.join(outdir, 'hparams.json'), 'w') as fout:
    json.dump(params, fout, indent=2)
with open(os.path.join(outdir, 'vocab.json'), 'w') as fout:
    json.dump(id2label, fout, indent=2)


# опишем функции, поставляющие данные в вычислительный граф
# см код в utils.py
train_input_fn = generator_input_fn(
    x=utils.fast_datagenerator(df_train, hparams, 'train'),
    target_key='target',
    batch_size=hparams.batch_size,
    shuffle=True,
    queue_capacity=3 * hparams.batch_size,
    num_threads=1,
)

val_input_fn = generator_input_fn(
    x=utils.fast_datagenerator(df_val, hparams, 'val'),
    target_key='target',
    batch_size=hparams.batch_size,
    shuffle=False,
    queue_capacity=3 * hparams.batch_size,
    num_threads=1,
)

In [ ]:
config = tf.estimator.RunConfig(model_dir=hparams.outdir)

est = tf.estimator.Estimator(
    model_fn=model_handler,
    config=config,
    params=hparams,
)

train_spec = tf.estimator.TrainSpec(input_fn=train_input_fn, max_steps=hparams.train_steps)
eval_spec = tf.estimator.EvalSpec(input_fn=val_input_fn)

tf.estimator.train_and_evaluate(est, train_spec, eval_spec)

Запустите tensorboard и смотрите графики обучения

```
tensorboard --logdir=./OUTDIR/   # в терминале на сервере
ssh -L 6006:localhost:6006 # в терминале у себя на машине
```
Зайдите браузером на http://localhost:6006


Теперь делаем предсказание.


In [ ]:
# готовим данные для теста из sample_submission
df = pd.read_csv(os.path.join(DATADIR, 'sample_submission.csv'))
df.label = 0
df.fname = [os.path.join(DATADIR, 'audio_test', _) for _ in df.fname.values]

# predict все равно работает по одному примеру, так что давайте уберем батчи
# так мы сможем работать с записями целиком
# NB: стоит проверить, правильно ли работает pad_value
test_input_fn = generator_input_fn(
    x=utils.fast_datagenerator(df, params, 'test'),
    batch_size=1,
    shuffle=False,
    num_epochs=1,
    queue_capacity=hparams.batch_size,
    num_threads=1,
    pad_value=0.0,
)

it = est.predict(input_fn=test_input_fn)  # это итератор

# идем по датасету и сохраняем вывод сетки
submission = dict()
for output in tqdm_notebook(it):
    path = output['fname'].decode()
    fname = os.path.basename(path)
    # допускается предсказывать три метки на каждую запись
    # см условие задачи и метрику оценки
    predicted = " ".join([id2label[i] for i in output['top3']])
    submission[fname] = predicted

# записываем в файл результаты
submission_path = os.path.join(outdir, 'submission.csv')
with open(submission_path, 'w') as fout:
    fout.write('fname,label\n')
    for fname, pred in submission.items():
        fout.write("{},{}\n".format(fname, pred))
        
print('Take you submission: {}'.format(submission_path))

**Модель с окном 8к тиков, тренировкой на 10k шагов показывает 0.4 MAP@3 и дает 0.538 на паблике**
